In [38]:
import ee
import geemap
import geopandas as gpd
import pandas as pd

from fwi_predict.geo.ee import get_sentinel2_l2a, get_nearest_sentinel2_image, scale_sentinel2_l2a

In [39]:
ee.Initialize(project='fwi-water-quality-sensing')

In [40]:
DEFAULT_BANDS = {'B1': 'aerosols',
                 'B2': 'blue',
                 'B3': 'green',
                 'B4': 'red',
                 'B5': 'red_edge_1',
                 'B6': 'red_edge_2',
                 'B7': 'red_edge_3',
                 'B8': 'NIR',
                 'B8A': 'red_edge_4',
                 'B9': 'water_vapor',
                 'B11': 'swir_1',
                 'B12': 'swir_2',
                 'AOT': 'aot',
                 'SCL': 'scene_classification',
                 'MSK_CLDPRB': 'cloud_probability'}

In [41]:
sentinel = get_sentinel2_l2a()

In [42]:
predict_ds = gpd.read_file("../data/clean/ara_predict_ds.geojson")
predict_ds_ee = geemap.gdf_to_ee(predict_ds)
predict_ds_ee = predict_ds_ee.map(lambda f: f.set('sample_dt', ee.Date(f.get('sample_dt')))) # Format dates as ee.Date

In [43]:
def get_sentinel2_values_at_feature(feature: ee.Feature) -> ee.Feature:
  """Write docstring later."""
  nearest_image = get_nearest_sentinel2_image(feature)

  # Select bands of interest and add vegetation indices
  nearest_image = scale_sentinel2_l2a(nearest_image)
  nearest_image = nearest_image.select(list(DEFAULT_BANDS.keys()))
  nearest_image = nearest_image.addBands([
    nearest_image.normalizedDifference(['B8', 'B4']).rename("ndvi"),
    nearest_image.normalizedDifference(['B3', 'B8']).rename('ndwi')
  ])

  values = nearest_image.sample(feature.geometry()) \
    .map(lambda f: f \
      .set('measurement_idx', feature.get('measurement_idx')) \
      .set('hours_from_measurement', nearest_image.get('hours_from_date'))
    )

  return values

In [44]:
result = predict_ds_ee.map(get_sentinel2_values_at_feature)

In [46]:
geemap.ee_export_vector_to_drive(result.flatten(), description='sentinel2_features', fileFormat='CSV')

Exporting sentinel2_features... Please check the Task Manager from the JavaScript Code Editor.
